In [9]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time
time1 = time.time()

store={}
item_uuids=[]
env = 'data'
my_auth = get_key('koray_data')
schema_name = get_schema_names(my_auth) 


set_url = '/search/?experiments_in_set.experiment_type=DNase+Hi-C&experimentset_type=replicate&lab.display_title=Chuck+Murry%2C+UW&type=ExperimentSetReplicate'
all_sets = ff_utils.search_metadata(set_url , key=my_auth)
ready_sets_1 = [i for i in all_sets if "HiC_Pipeline_0.2.5" in i.get('completed_processes', [])]

set_to_release = [i['uuid'] for i in ready_sets_1]
print(len(set_to_release))

store, uuids = record_object_es(set_to_release, my_auth, schema_name, add_pc_wfr=True, store_frame='object', ignore_field = ['experiment_relation', 'biosample_relation', 'references'])
print(len(store['experiment_set_replicate']))
print(len(uuids))
print(len(list(set(uuids))))
time2 = time.time()
print((time2-time1))

8
8
801
801
12.017963171005249


In [10]:
print(set_to_release)
print([i['uuid'] for i in store['experiment_set_replicate']])

['b5a53608-aced-4dfc-ae64-2a78ceed902b', 'f07ef94b-a623-4833-b9b3-676b7d7e75ab', '3e481c80-022e-4ee6-9efe-8f9df6bd50fa', '897038d2-4267-4bbb-89b4-60201a7bf337', '5db537d2-1cf6-44c2-ae5d-6d724c9f43bc', '2a9d5a2d-f469-43ef-8f71-2cca3c872156', '208dc2d9-549c-4533-9f77-01e0f0490755', '37ac19c8-3280-4026-91ba-e401b4ffa1e9']
['f07ef94b-a623-4833-b9b3-676b7d7e75ab', 'b5a53608-aced-4dfc-ae64-2a78ceed902b', '897038d2-4267-4bbb-89b4-60201a7bf337', '5db537d2-1cf6-44c2-ae5d-6d724c9f43bc', '3e481c80-022e-4ee6-9efe-8f9df6bd50fa', '37ac19c8-3280-4026-91ba-e401b4ffa1e9', '2a9d5a2d-f469-43ef-8f71-2cca3c872156', '208dc2d9-549c-4533-9f77-01e0f0490755']


In [11]:
# TODO
# Check audits

# do you want to check for duplicate/problematic runs on files?
# it will take some time
check_wfrs = True
delete_problematic = False

# check expsets
print('EXPSET CHECK')
for a_set in store['experiment_set_replicate']:
    if not a_set.get('completed_processes'):
        print(a_set['accession'], 'missing processing tag', a_set['description'][:50])

# check exps 
print('\nEXP CHECK')
# check for experiment numbers
exp_names = ['experiment_hi_c', 'experiment_mic', 'experiment_seq']
all_exps_on_sets = [a for i in store['experiment_set_replicate'] for a in i['experiments_in_set']]
all_exps = [a['uuid'] for i in store.keys() for a in store[i] if i in exp_names]
if len(all_exps_on_sets) != len(all_exps):
    print('Number of experiments is not same as experiments associated with sets')
    print('# of exps: {}. # of exps on sets: {}'.format(len(store['experiment_hi_c']), len(all_exps_on_sets)))
        
print('\nFILE FASTQ CHECK')
for a_file in store['file_fastq']:
    if not a_file.get('quality_metric'):
        print(a_file['accession'], 'missing fastqc')
    if not a_file.get('content_md5sum') or not a_file.get('md5sum'):
        print(a_file['accession'], 'problems with md5')
    if check_wfrs:
        dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)

# check processed files
print('\nFILE PROCESSED CHECK')
if store.get('file_processed'):
    for a_file in store['file_processed']:
        if a_file['file_format'] == '/file-formats/pairs/':
            if not a_file.get('quality_metric'):
                print(a_file['accession'], 'missing Pairsqc')
        if not a_file.get('source_experiments'):
            print(a_file['accession'], 'missing source experiments')
        if check_wfrs:
            dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)   

# check wfrs
print('\nWFR CHECK')
if store.get('workflow_run_awsem'):
    for wfr in store['workflow_run_awsem']:
        if wfr['run_status'] != 'complete':
            print('problematic wfr', wfr['uuid'], wfr['run_status'])
        
# check for weird status
print('\nREPORT NUMBERS AND CHECK STATUS')
for i in store:
    print(i, len(store[i]))
    weird = [[i,x['uuid'],x['status']] for x in store[i] if x['status'] in ['deleted', 'replaced', 'archived', 'revoked']]
    if weird:
        for case in weird:
            print(case)
        print()


EXPSET CHECK

EXP CHECK

FILE FASTQ CHECK

FILE PROCESSED CHECK

WFR CHECK

REPORT NUMBERS AND CHECK STATUS
file_reference 2
enzyme 1
user 14
vendor 3
file_processed 110
static_section 1
workflow 7
file_format 14
experiment_hi_c 16
individual_human 3
ontology 4
biosource 3
ontology_term 67
software 8
quality_metric_pairsqc 24
experiment_set_replicate 8
protocol 11
organism 1
biosample 16
workflow_run_awsem 278
lab 7
quality_metric_fastqc 92
award 5
file_fastq 92
biosample_cell_culture 14


In [13]:
# Check status
change_status = 'released'

common_types = ['lab', 'user', 'award','ontology', 'ontology_term', 
                'file_format', 'software', 'workflow', 'static_section',
                'organism', 'vendor', 'file_reference', 'individual_human', 
                'enzyme', 'biosource', 'file_format']

print('COMMON ITEMS')
for a_type in store:
    if a_type in common_types:
        for raw_data in store[a_type]:
            if raw_data['status'] not in ['released', 'current']:
                print('COMMON ITEM NOT RELEASED', a_type,raw_data['status'], raw_data['uuid'])
                # ff_utils.patch_metadata({'status':'released'}, obj_id=raw_data['uuid'] ,key=my_auth)
print('---------------------------------\n')

counter = 0
for a_type in store:
    if a_type in common_types:
        continue
    for raw_data in store[a_type]:
        if raw_data['status'] == 'released':
            print(a_type, raw_data['uuid'], raw_data['status'])
            continue
        if raw_data['status'] != change_status:
            print(a_type, raw_data['uuid'], raw_data['status'])
            # ff_utils.patch_metadata({'status':'released to project'}, obj_id=raw_data['uuid'] ,key=my_auth)
            



COMMON ITEMS
---------------------------------

file_processed fe9e23ee-494e-4f6e-a85c-ca148dd040e8 released
file_processed ed802b5a-6372-431d-8d3b-c76f4e809e18 uploaded
file_processed ce7d1f9d-2a3b-4b57-9ba4-2aae7c0d39ba released
file_processed abae9e9e-a44e-44f6-b42d-c7b4bd6b56fc released
file_processed a085ca19-7aaf-4df9-a81c-6d6a0bf88bda released
file_processed 930023f9-af8d-4141-bd2e-3c0049040203 uploaded
file_processed 82b3f337-65d5-4ddb-b8b1-d4d471b22562 uploaded
file_processed 72de82f0-48a9-4809-bf5c-e4ae4e2e23a9 uploaded
file_processed 71e7e10a-53b7-4c86-9b02-264680db4caf uploaded
file_processed 6e4727de-0085-4b35-bece-6fa69663f4e6 released
file_processed 65d32ce2-c504-4b15-90ca-b326731ea41b released
file_processed 5c5e7a92-5d46-4d04-9662-9e988367c996 released
file_processed 5aa5c550-e300-44c9-ba60-887065c3d8bc released
file_processed 4a366884-a1ef-42ec-b617-6413c552bbb3 uploaded
file_processed 48b00aed-bd4f-4e27-ae00-db2f4cb5c53a uploaded
file_processed 3ca8628d-fa37-4c1e-bd9

file_fastq 05212134-794c-4928-89ca-79756d0256fe released
file_fastq 04b1d0e9-5bd3-4cd3-85d6-29118a73c5ab released
file_fastq 00ae2617-37cf-485f-a807-06131f406d04 released
biosample_cell_culture 50eae307-a9db-4a93-bf25-ee275f8ba584 released
biosample_cell_culture 0b6069be-8d65-4c0d-8dab-5efa26f87b08 released
biosample_cell_culture 8fe6da73-7e0b-4220-a28b-a439589d9277 released
biosample_cell_culture 665b873f-e154-4f70-8a5d-dc1c3ac1b35d released
biosample_cell_culture 2f5e738e-2b13-49d4-aa96-a8467f74d200 released
biosample_cell_culture 2164eb9b-6df1-4bd2-b9b7-b9a8bcf42ef0 released
biosample_cell_culture d22a1796-015e-4b5c-9c0f-b7cee8181a64 released
biosample_cell_culture b85b19b8-1f91-4cac-b58e-0f43e4d4c647 released
biosample_cell_culture 35f63752-f6f0-495e-a936-0d39e265fa3e released
biosample_cell_culture 30d6baf6-d1ff-4119-8e41-b3d8f0d767e6 released
biosample_cell_culture f631daba-02b0-4a5f-89e6-955eeb8e9679 released
biosample_cell_culture 5f7ec8a9-233a-4fc7-8f50-3c9293da9562 released
b

In [14]:
# Release the items
action = True
change_status = 'released'

counter = 0
for a_type in store:
    if a_type in common_types:
        continue
    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print('deleted data', a_type ,raw_data['uuid'])
            continue        
        if raw_data['status'] not in ['current', 'released']:    
            counter +=1
            print(a_type, raw_data['status'], raw_data['uuid'])
            patch_data = {'status': change_status}
            if action:
                if change_status == 'released':
                    if a_type in ['publication', 'user', 'award', 'vendor', 'organism', ]:
                        ff_utils.patch_metadata({'status': 'current'}, obj_id=raw_data['uuid'] ,key=my_auth)
                    else:
                        ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)
                else:
                    ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)
                
        else:
            print('ALREADY RELEASED/TO PROJECT', a_type, raw_data['status'], raw_data['uuid'])
            pass
            
print(counter)

ALREADY RELEASED/TO PROJECT file_processed released fe9e23ee-494e-4f6e-a85c-ca148dd040e8
file_processed uploaded ed802b5a-6372-431d-8d3b-c76f4e809e18
ALREADY RELEASED/TO PROJECT file_processed released ce7d1f9d-2a3b-4b57-9ba4-2aae7c0d39ba
ALREADY RELEASED/TO PROJECT file_processed released abae9e9e-a44e-44f6-b42d-c7b4bd6b56fc
ALREADY RELEASED/TO PROJECT file_processed released a085ca19-7aaf-4df9-a81c-6d6a0bf88bda
file_processed uploaded 930023f9-af8d-4141-bd2e-3c0049040203
file_processed uploaded 82b3f337-65d5-4ddb-b8b1-d4d471b22562
file_processed uploaded 72de82f0-48a9-4809-bf5c-e4ae4e2e23a9
file_processed uploaded 71e7e10a-53b7-4c86-9b02-264680db4caf
ALREADY RELEASED/TO PROJECT file_processed released 6e4727de-0085-4b35-bece-6fa69663f4e6
ALREADY RELEASED/TO PROJECT file_processed released 65d32ce2-c504-4b15-90ca-b326731ea41b
ALREADY RELEASED/TO PROJECT file_processed released 5c5e7a92-5d46-4d04-9662-9e988367c996
ALREADY RELEASED/TO PROJECT file_processed released 5aa5c550-e300-44c9-b

ALREADY RELEASED/TO PROJECT quality_metric_pairsqc released 72e243b9-e6fe-4cb8-a85a-0f4ca66472db
ALREADY RELEASED/TO PROJECT quality_metric_pairsqc released ee79bd4b-d887-48ca-9144-f59534258f9a
ALREADY RELEASED/TO PROJECT quality_metric_pairsqc released e2f38542-c66b-4ad3-ab77-0481607c8f36
quality_metric_pairsqc in review by lab 29d2e0b9-9c09-4e21-b4e0-f7b4a6ff1d24
ALREADY RELEASED/TO PROJECT quality_metric_pairsqc released 1889a0a9-2e83-496f-87a8-34e8d33a6526
quality_metric_pairsqc in review by lab 14d80f77-a4ee-42e1-8bfc-ceb199532ce7
quality_metric_pairsqc in review by lab a25e2b93-25be-498f-87ee-3474e6adcbc9
ALREADY RELEASED/TO PROJECT quality_metric_pairsqc released 6f0b8fe1-173a-4f17-a14e-212dcdac1624
ALREADY RELEASED/TO PROJECT quality_metric_pairsqc released 567bf013-cffc-41da-84be-31e24732b094
ALREADY RELEASED/TO PROJECT quality_metric_pairsqc released 46e6bceb-aadc-4e27-8f96-304b8c8170b6
quality_metric_pairsqc in review by lab 03f4f705-8117-47af-859d-7f8a8dd21ce9
ALREADY RELEA

workflow_run_awsem in review by lab 6c1f6a1e-21cc-4c30-99c7-198b331e8b2e
workflow_run_awsem in review by lab 6888ea3e-19a8-49fb-970f-89c768ac57cb
workflow_run_awsem in review by lab 66e797f0-3758-4eaf-a1c0-59c22189fdd7
workflow_run_awsem in review by lab 5d2c07d9-ecac-4b25-a8b6-202044ce8bff
workflow_run_awsem in review by lab 5d2341a5-07c6-4cfd-9f21-b730e7d893b7
workflow_run_awsem in review by lab 5bddc4ff-94da-4e42-a144-30c930889d55
workflow_run_awsem in review by lab 58b12974-d5ae-4008-a1bc-8fd6e34636ad
ALREADY RELEASED/TO PROJECT workflow_run_awsem released 58ad097d-7c61-41e6-962f-025b0a285443
workflow_run_awsem in review by lab 57d3c17d-d19b-4bf5-a285-76ab1179c8d7
workflow_run_awsem in review by lab 515912b2-f1b0-4ef7-8889-f118f7e813b4
workflow_run_awsem in review by lab 514ab023-6d94-4de8-b52f-3770b7b87e6d
ALREADY RELEASED/TO PROJECT workflow_run_awsem released 437ffa0c-551d-47b3-a8ee-97b1b44ddd08
ALREADY RELEASED/TO PROJECT workflow_run_awsem released 41a376dc-d84e-40d2-9090-a300

workflow_run_awsem in review by lab c638f96b-77d1-4413-9b3a-737efe7b748b
workflow_run_awsem in review by lab b9038054-6daf-4853-b35b-6429b102f7b8
workflow_run_awsem in review by lab b5a1b633-91eb-4908-937d-5a1d56fe0fe5
workflow_run_awsem in review by lab b526f5e1-e8e2-4bb5-a4be-0d00711c6ce2
workflow_run_awsem in review by lab b1bdd27a-7ea3-452d-ac5b-736a655eafaa
workflow_run_awsem in review by lab a8553aea-c47b-4744-94f4-111f288b09ce
workflow_run_awsem in review by lab a5303f9d-1566-492a-ba07-1698e1fdf5f1
workflow_run_awsem in review by lab a396301c-bbbf-4a75-8027-2434b5b60aed
ALREADY RELEASED/TO PROJECT workflow_run_awsem released a003e574-c2fd-42d4-a873-1ce02e71b006
workflow_run_awsem in review by lab 9cca1331-9f8d-4c4e-8b67-a583fbac2202
workflow_run_awsem in review by lab 98b2514e-acc6-4da9-8bc4-212577823e55
workflow_run_awsem in review by lab 9573c8a2-311a-41f8-8e6e-0b4ecb4e87f6
workflow_run_awsem in review by lab 94c28316-bd31-43aa-99e1-3624108e114d
workflow_run_awsem in review by

workflow_run_awsem in review by lab d1e8ad76-5a5a-4ff0-827b-64b9283e16a8
ALREADY RELEASED/TO PROJECT workflow_run_awsem released cf284382-ff44-4d14-bd54-53f06bdf247e
ALREADY RELEASED/TO PROJECT workflow_run_awsem released c7722ad1-44e0-4428-a263-0408beb662ab
ALREADY RELEASED/TO PROJECT workflow_run_awsem released bf6d45c1-de93-4ef0-999f-60feba929468
workflow_run_awsem in review by lab ae0d1f5b-7820-4bbf-8a03-aade78bd4b17
workflow_run_awsem in review by lab a9395375-4e6c-42e5-8357-272b4295a106
workflow_run_awsem in review by lab a5019c23-0413-430c-bee0-b937f7f067b2
workflow_run_awsem in review by lab a368ecad-2ab1-4a8b-ab25-f25c406f1978
workflow_run_awsem in review by lab 9383198c-ece0-4ae9-8758-1fa99f02fe18
workflow_run_awsem in review by lab 87303f40-7735-4d51-9f2e-53c860168791
workflow_run_awsem in review by lab 87178b20-6741-4b89-a3f9-62968f41f27f
workflow_run_awsem in review by lab 6f6bdcb8-383e-4d20-8841-e0b30e8fd17d
ALREADY RELEASED/TO PROJECT workflow_run_awsem released 6d359c3a

ALREADY RELEASED/TO PROJECT biosample_cell_culture released 30d6baf6-d1ff-4119-8e41-b3d8f0d767e6
ALREADY RELEASED/TO PROJECT biosample_cell_culture released f631daba-02b0-4a5f-89e6-955eeb8e9679
ALREADY RELEASED/TO PROJECT biosample_cell_culture released 5f7ec8a9-233a-4fc7-8f50-3c9293da9562
ALREADY RELEASED/TO PROJECT biosample_cell_culture released 48a30367-0b69-475a-9a53-9807e45567ee
ALREADY RELEASED/TO PROJECT biosample_cell_culture released e304a46d-a03a-4865-9c82-3e8f34f5b5ad
264
